In [ ]:
# This notebook classifies the generation of Pokemon based on their images
# JM
# ANLY 590 Final Project
import numpy as np 
import pandas as pd 

import tensorflow as tf 
import re
from keras import backend as K 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import tensorflow.keras.layers as L 
from tensorflow.keras.optimizers import Adam 

from tensorflow.keras.losses import SparseCategoricalCrossentropy 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing import image 
from PIL import Image 

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization 
from google.colab import drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Pokemon/Pokemon/

/content/drive/My Drive/Pokemon/Pokemon


In [ ]:
# read in the data
pokemons = pd.read_csv('pokemon-images-and-types/pokemon.csv')
numbers = []
for i in range(1,pokemons.shape[0]+1):
    numbers.append(i)
pokemons['pkn'] = numbers
IMG_DIR = 'pokemon-images-dataset/pokemon'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]

In [ ]:
# Add generations for each pokemon
gen = [] 
length = len(pokemons) 

for i in range(0, length):
    if i<=150:
        gen.append(1)
    elif i<=250:
        gen.append(2)
    elif i<=385:
        gen.append(3)
    elif i<=492:
        gen.append(4)
    elif i<=648:
        gen.append(5)
    elif i<=720:
        gen.append(6)
    else:
        gen.append(7)
        

        
pokemons["Gen"] = gen 

pokemons["Gen"] = pokemons["Gen"].astype(str) 



pokemons.head() 

,Name,Type1,Type2,pkn,Gen
0,bulbasaur,Grass,Poison,1,1
1,ivysaur,Grass,Poison,2,1
2,venusaur,Grass,Poison,3,1
3,charmander,Fire,NaN,4,1
4,charmeleon,Fire,NaN,5,1


In [ ]:
numbers = []
for i in range(1,pokemons.shape[0]+1):
    numbers.append(i)
pokemons['pkn'] = numbers
IMG_DIR = 'pokemon-images-dataset/pokemon'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]

In [ ]:
import re
dataframe_img = pd.DataFrame([])
images = []
pokemon_number = []
for img in onlyfiles:
    if not re.search('-', img):
        pkn = img.split('.')
        n = re.sub("[^0-9]", "", pkn[0])
        path = IMG_DIR +'/' +str(img)
        images.append(path)
        pokemon_number.append(n)
dataframe_img['images'] = images
dataframe_img['pkn'] = pokemon_number
dataframe_img['pkn'] = dataframe_img['pkn'].astype(int)
dataframe_img['pkn'] = dataframe_img['pkn'].astype(int)
result = pokemons.merge(dataframe_img, left_on='pkn', right_on='pkn')
result.head()

,Name,Type1,Type2,pkn,Gen,images
0,bulbasaur,Grass,Poison,1,1,pokemon-images-dataset/pokemon/1.png
1,ivysaur,Grass,Poison,2,1,pokemon-images-dataset/pokemon/2.png
2,venusaur,Grass,Poison,3,1,pokemon-images-dataset/pokemon/3.png
3,charmander,Fire,NaN,4,1,pokemon-images-dataset/pokemon/4.png
4,charmeleon,Fire,NaN,5,1,pokemon-images-dataset/pokemon/5.png


In [ ]:
# split training and test set
imageTrain, imageTest, genTrain, genTest = train_test_split(result["images"], result["Gen"], test_size = 0.3, random_state = 919)

imageTrain, imageVal, genTrain, genVal = train_test_split(imageTrain, genTrain, test_size = 0.4, random_state = 919)

In [ ]:
datagen = image.ImageDataGenerator()
trainDf = pd.DataFrame(imageTrain) 
trainDf["Gen"] = genTrain 
train = datagen.flow_from_dataframe(trainDf, x_col = "images", y_col = "Gen") 

Found 297 validated image filenames belonging to 6 classes.


In [ ]:
testDf = pd.DataFrame(imageTest) 
testDf["Gen"] = genTest 
test = datagen.flow_from_dataframe(testDf, x_col = "images", y_col = "Gen") 

Found 213 validated image filenames belonging to 6 classes.


In [ ]:
valDf = pd.DataFrame(imageVal) 
valDf["Gen"] = genVal 
val = datagen.flow_from_dataframe(valDf, x_col = "images", y_col = "Gen") 

Found 198 validated image filenames belonging to 6 classes.


In [ ]:

 
WIDTH = 256 
HEIGHT = 256
CHANNELS = 3 

model = Sequential() 
model.add(L.Lambda(lambda x: x, input_shape = (WIDTH, HEIGHT, CHANNELS))) 
model.add(Conv2D(32, (2, 2))) 
model.add(BatchNormalization()) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(32, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(64, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Conv2D(256, (2, 2))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten()) 
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid')) 

model.compile(loss = 'binary_crossentropy', 
              optimizer='Adam', metrics=['acc'])

history = model.fit(train, epochs = 50, validation_data=val) 

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
10/10 [==============================] - 27s 3s/step - loss: 0.8904 - acc: 0.5707 - val_loss: 18.6599 - val_acc: 0.1667
Epoch 2/50
10/10 [==============================] - 28s 3s/step - loss: 0.7880 - acc: 0.5258 - val_loss: 0.7578 - val_acc: 0.8232
Epoch 3/50
10/10 [==============================] - 26s 3s/step - loss: 0.7210 - acc: 0.5864 - val_loss: 0.6352 - val_acc: 0.7593
Epoch 4/50
10/10 [==============================] - 26s 3s/step - loss: 0.7279 - acc: 0.5640 - val_loss: 0.6532 - val_acc: 0.8165
Epoch 5/50
10/10 [==============================] - 26s 3s/step - loss: 0.6761 - acc: 0.6044 - val_loss: 0.9936 - val_acc: 0.4226
Epoch 6/50
10/10 [==============================] - 26s 3s/step - loss: 0.6263 - acc: 0.6627 - val_loss: 3.1322 - val_acc: 0.1667
Epoch 7/50
10/10 [==============================] - 26s 3s/step - loss: 0.6167 - acc: 0.6740 - val_loss: 2.5832 - val_acc: 0.1700
Epoch 8/50
10

In [ ]:
predict = model.predict(test)

Instructions for updating:
Please use Model.predict, which supports generators.


In [ ]:
loss, accuracy = model.evaluate(test)
print(accuracy)

7/7 [==============================] - 4s 606ms/step - loss: 0.4517 - acc: 0.8333
0.8333333134651184
